In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.framework import ops
import imghdr
import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib
from google.protobuf import text_format

import os

print(tf.__version__)

ModuleNotFoundError: No module named 'freeze_graph'

In [31]:
os.listdir('./inputs')

['essex-face-recognition-dataset.zip', 'faces94']

In [32]:
picture_paths = []
labels        = []
main_dir = '../data/faces94/'

for sex_dir in os.listdir(main_dir):
    for person_dir in os.listdir(main_dir + '/' + sex_dir):
        for picture in os.listdir(main_dir + '/' + sex_dir + '/' + person_dir):
            if imghdr.what(main_dir + '/' + sex_dir + '/' + person_dir + '/' + picture) == 'jpeg':
                labels.append(person_dir)
                picture_paths.append(main_dir + '/' + sex_dir + '/' + person_dir + '/' + picture)

In [33]:
len(picture_paths)

2793

In [34]:
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=3)
    image_decoded = tf.reshape(image_decoded, [200*180*3])
    image_resized = image_decoded/255
    return image_resized, label

In [35]:
dataset = tf.data.Dataset.from_tensor_slices((picture_paths, labels))
dataset = dataset.map(_parse_function)

In [36]:
#dataset = dataset.batch(800)
#itr = dataset.make_one_shot_iterator()
#next_itr = itr.get_next()

In [37]:
#init = tf.global_variables_initializer()
#with tf.Session() as sess:
#    sess.run(init)
#    for i in range(2):
#        x,y = sess.run(next_itr)
#        print(x.shape)
#        fig = plt.figure(figsize=(5,5)); plt.axis('off')
#        plt.imshow(x); plt.show()

In [38]:
def create_placeholder(batch_size):
    X = tf.placeholder(shape = [None, 200*180*3], dtype = 'float' , name = 'input_node')
    Y = tf.placeholder(shape = [batch_size]     , dtype = 'string', name = 'Y'         )
    return X, Y

In [39]:
def initialize_parameters():           
    W1 = tf.get_variable('W1', [3, 3, 3, 6], initializer=tf.contrib.layers.variance_scaling_initializer())
    W2 = tf.get_variable('W2', [3, 3, 6, 6], initializer=tf.contrib.layers.variance_scaling_initializer())
    W3 = tf.get_variable('W3', [3, 3, 6, 6], initializer=tf.contrib.layers.variance_scaling_initializer())
    
    parameters = {'W1' : W1,
                  'W2' : W2,
                  'W3' : W3 }
    
    return parameters

In [40]:
def first_layer_forward(X, parameters):
    W1 = parameters['W1']
    
    X = tf.reshape(X, [-1, 200, 180, 3], name='X')
    z = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
    a = tf.nn.elu(z)
    p = tf.nn.max_pool(a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    return p

In [41]:
def generic_skip_layer(p, parameters, num_layer):
    W = parameters['W' + str(num_layer)]
    
    z = tf.nn.conv2d(p, W, strides = [1,1,1,1], padding='SAME')
    a = tf.nn.elu(z)# + a_skip)
    p = tf.nn.max_pool(a, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    return p

In [42]:
def flatten_layer(p, keep_prob=True):
    flat = tf.contrib.layers.flatten(p)
    
    dense    = tf.layers.dense(inputs=flat, units = 1024, activation=tf.nn.elu)
    dropout  = tf.layers.dropout(inputs=dense, rate=0.4, training=keep_prob)
    
    return dropout

In [43]:
def encoding_layer(dropout):
    encoding = tf.contrib.layers.fully_connected(inputs=dropout, num_outputs=128, activation_fn=None)
    return encoding

In [44]:
def freeze_my_graph(sess):    
    input_graph_path            = './tmp/input_model/input_graph.pbtxt'
    input_saver_def_path        = None
    input_binary                = False
    input_checkpoint_path       = './tmp/triplet_mode.ckpt'
    output_node_names           = 'fully_connected/BiasAdd'
    restore_op_name             = 'save/restore_all'
    filename_tensor_name        = 'save/Const:0'
    output_graph_path           = './tmp/input_model/output_graph.pb'
    output_optimized_graph_name = './tmp/input_model/optimized_graph.pb'
    clear_devices               = True
    initializer_nodes           = ''
    
    freeze_graph.freeze_graph(input_graph_path     ,     
                              input_saver_def_path , 
                              input_binary         ,         
                              input_checkpoint_path,
                              output_node_names    ,    
                              restore_op_name      ,      
                              filename_tensor_name , 
                              output_graph_path    ,    
                              clear_devices        ,
                              initializer_nodes    )
    
    gd = tf.GraphDef()
    
    with tf.gfile.Open(output_graph_path, 'rb') as f:
        gd.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
        gd,
        ['input_node'],
        [output_node_names],
        tf.float32.as_datatype_enum)
    
    with tf.gfile.FastGFile(output_optimized_graph_name, 'wb') as f:
        f.write(output_graph_def.SerializeToString())

In [73]:
def model_facenet(picture_paths, labels, learning_rate = 0.0001, num_epochs = 5, batch_size=32, margin=0.2):
    ops.reset_default_graph()
    m = 2796
    #(m, n_h, n_w, n_c) = dataset.shape
    encodings = []
    costs     = []
    
    dataset = tf.data.Dataset.from_tensor_slices((picture_paths, labels))
    dataset = dataset.map(_parse_function)
    
    test_dataset  = dataset.take(200)
    train_dataset = dataset.skip(200)
    
    test_faces  = []
    test_labels = []
    test_itr    = test_dataset.make_one_shot_iterator()
    next_test   = test_itr.get_next()
    
    train_dataset = train_dataset.batch(batch_size)
    kp            = tf.placeholder(dtype='bool', name='kp')

    init_itr  = train_dataset.make_initializable_iterator()
    next_init = init_itr.get_next()

    faceholder, labelholder = create_placeholder(batch_size)
    parameters = initialize_parameters()
    
    #------LOOP FOR ANCHOR------#
    first_hidden_layer = first_layer_forward(faceholder, parameters)
    
    pool = generic_skip_layer(first_hidden_layer, parameters,  2)
    pool = generic_skip_layer(pool              , parameters,  3)

    flat     = flatten_layer (pool, kp)
    encoding = encoding_layer(flat)
    
    #------BACKPROPOGATION FOR WEIGHT UPDATES------#
    cost      = tf.contrib.losses.metric_learning.triplet_semihard_loss(labels=labelholder, embeddings=encoding, margin=margin)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    #Saver for checkpointing model training & reloading
    saver = tf.train.Saver()
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        tf.train.write_graph(sess.graph.as_graph_def(), './tmp/input_model', 'input_graph.pbtxt', True)
        for epoch in range(num_epochs):
            sess.run(init_itr.initializer)
            epoch_cost  = 0
            num_batches = (int)((m-200) / batch_size)
            for batch in range(num_batches):
                faces, labels = sess.run(next_init)
                _, batch_cost = sess.run([optimizer, cost], feed_dict = { faceholder : faces, labelholder : labels, kp : True})
                epoch_cost += batch_cost / num_batches
            
            if epoch % 10 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                saver.save(sess, './tmp/triplet_mode.ckpt')
        
        for i in range(200):
            test_x, test_y = sess.run(next_test)
            test_faces .append(test_x)
            test_labels.append(test_y)
        
        test_pred = sess.run(encoding, feed_dict={faceholder : test_faces, kp : False})
        return test_pred, test_labels

In [74]:
test_pred, test_labels =  model_facenet(picture_paths=picture_paths, labels=labels, num_epochs=1, margin=0.2)

Cost after epoch 0: 0.049557


In [91]:
face  = tf.read_file('C:/facial_recognition/inputs/faces94/male/ambarw/ambarw.12.jpg')
face = tf.image.decode_jpeg(face, channels=3)
face = tf.reshape(face, [1, 200*180*3])
face = face/255

saver = tf.train.import_meta_graph('./tmp/triplet_mode.ckpt.meta')

with tf.Session() as sess:
    saver.restore(sess, './tmp/triplet_mode.ckpt')
    n_face = face.eval()
    print(n_face)
    logits = sess.run('fully_connected/BiasAdd:0', feed_dict={'input_node:0' : n_face, 'kp:0': False})
    
    #freeze_my_graph(sess)        
    #best_theta = theta.eval()

INFO:tensorflow:Restoring parameters from ./tmp/triplet_mode.ckpt
[[0.48627454 0.7411765  0.33333334 ... 1.         0.8588236  0.62352943]]


In [75]:
correct = 0
for i in range(200):
    j = i + 1
    while j < 200:
        diff = np.linalg.norm(test_pred[i] - test_pred[j])
        if (diff < 2.0 and test_labels[i] == test_labels[j]) or (diff >= 2.0 and test_labels[i] != test_labels[j]):
            correct += 1
            
        j+=1
print(correct/19900)

0.9571859296482412
